# Motivation

Perhaps a more practice-based approach could be more fruitful. can use textbook & images as RAG dataset.

# Dependencies

In [2]:
!pip install -U datasets pinecone-client sentence-transformers torch

  Using cached datasets-2.14.6-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_client-2.2.4-py3-none-any.whl.metadata (7.8 kB)
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch-2.1.0-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Using cached dnspython-2.4.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached scikit_learn-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.11.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 2.4 MB/s eta 0:00:002.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 1.2 MB/s eta 0:00:001.2 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d0923deadf0c81b32b71c27a32f10cc6c213f666777e90582bbaec797002ec79
  Stored in directory: /home/developer/.cache/pip/wheels/ff/27/bf/ffba8b318b02d7f691a57084ee154e26ed24d012b0c7805881
Successfully built sentence-t

# Dataset

In [3]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
radiology_data = load_dataset(
    path = 'Ka4on/radiology',
    split='train',
    streaming=True
).shuffle(seed=960)

/home/developer/anaconda3/envs/andy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# streaming mode allows us to iterate over the dataset without needing to download it
# show the contents of a single document in the dataset
next(iter(radiology_data))

{'instruction': 'Generate impression based on medical findings.',
 'input': 'Dysarthria. There is no evidence of intracranial hemorrhage, mass, or acute infarct. There are mild scattered foci of cerebral white matter T2 hyperintensity. There is diffuse cerebral volume loss, which is most pronounced in the medial temporal lobes. There is no midline shift or herniation. The major cerebral flow voids are intact. The orbits, skull, paranasal sinuses, and scalp soft tissues are grossly unremarkable.',
 'output': '1. Nonspecific mild scattered foci of cerebral white matter T2 hyperintensity may represent chronic small vessel ischemic disease. Otherwise, no evidence of acute infarction.2. Diffuse cerebral volume loss, which is most pronounced in the medial temporal lobes, which may represent Alzheimer disease in the appropriate clinical setting. '}

In [5]:
from tqdm.auto import tqdm

total_prognosis_count = 50000

counter = 0
docs = []

for d in tqdm(radiology_data, total=total_prognosis_count):
    # extract the fields we need
    doc = {
        "input": d["input"],
        "output": d["output"],
    }
    docs.append(doc)

    # stop iteration once we reach 50k
    if counter == total_prognosis_count:
        break
    counter += 1

100%|██████████████████████████████████| 50000/50000 [00:04<00:00, 10365.67it/s]


In [6]:
import pandas as pd

df = pd.DataFrame(docs)
df.head()

,input,output
0,Dysarthria. There is no evidence of intracrani...,1. Nonspecific mild scattered foci of cerebral...
1,Male 8 years old Reason: Ao root dilatation Le...,1. Status post arterial switch operation.2. No...
2,Pituitary adenoma status post TSH in 11/2013: ...,Interval evolution postoperative findings rela...
3,"History of neuroblastoma of lumbar spine, rela...",1. Postoperative findings related to laminecto...
4,"Encephalopathy: confusion, encephalopathy. Man...",Scattered chronic infarcts and probable chroni...


# Initialize Pinecone idx

In [7]:
import pinecone

# connect to pinecone environment
pinecone.init(
    api_key="05ce3e92-d0da-4ef4-9e3b-2a1ad822689b",
    environment="us-east1-gcp"  # find next to API key in console
)

In [8]:
index_name = "qa"

# check if the abstractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

# connect to abstractive-question-answering index we created
index = pinecone.Index(index_name)

# Initialize Retriever

##### Retriever Tasks:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. 

Uses SentenceTransformer model based on Microsoft's MPNet as our retriever.

In [9]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# load the retriever model from huggingface model hub
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

/home/developer/anaconda3/envs/andy/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


cpu


/home/developer/anaconda3/envs/andy/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

# Generate Embeddings and Upsert

In [10]:
# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch["input"].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

100%|█████████████████████████████████████████| 782/782 [30:06<00:00,  2.31s/it]


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 50001}},
 'total_vector_count': 50001}

# Initialize Generator

In [11]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
# TODO: can use LLM trained on textbook, but using RAG with textbooks may be more effective
# going to write down thought process and steps later in documentation
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

In [17]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    return xc

def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['input']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

In [24]:
# example
query = "are there any image findings that are particularly indicative of a positive or negative prognosis for patients with lung cancer?"

context = query_pinecone(query, top_k=1)
print(context)

query = format_query(query, context['matches'])
print(query)

{'matches': [{'id': '39306',
              'metadata': {'input': 'Evaluate disease status for clinical '
                                    'trial. Provide 3-D measurements are all '
                                    'reference lesions and compare prior CT. '
                                    'Lung cancer status post 22 cycles of '
                                    'chemotherapy. CHEST:LUNGS AND PLEURA: '
                                    'Right upper lobe mass measures 2.3 x 1.8 '
                                    'cm (image 30; series 4), not '
                                    'significantly changed from previous '
                                    'study.MEDIASTINUM AND HILA: Small '
                                    'pericardial effusion. Stable calcified '
                                    'right hilar and subcarinal lymph '
                                    'nodes.CHEST WALL: No significant '
                                    'abnormality notedABDOMEN:LIVE

In [25]:
# TODO: would insert RAG functionality here - explore how!

# Testing / Eval

In [26]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return print(answer)

In [27]:
query = "are there any image findings that are particularly indicative of a positive or negative prognosis for patients with lung cancer?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

There are a couple of things to consider. The first is that there is a lot of variability in the image quality. The second is that there is a lot of variability in the image quality


# Appendix

### source / references
- https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/question-answering/abstractive-question-answering.ipynb#scrollTo=6iFLxPPvx2Tm